# What

This is going to add the content extraction of the reports form atsb and tsbi nto the engine.

It carries on the work from #257 of adding pdf text extraction to the engine.

In [ ]:
import pandas as pd
from plotnine import ggplot, geom_histogram, aes, facet_wrap, geom_bar

import engine.extract.ReportExtracting as ReportExtracting

import importlib
import os
import shutil

importlib.reload(ReportExtracting)

Here I will setup some of the global values needed for this workbook.

In [ ]:
output_folder = '../../output'
report_text = pd.read_pickle(os.path.join('converted_with_headers.pkl'))
report_text['year'] = report_text['report_id'].str.extract('(\d{4})').astype(int)
report_text['mode'] = report_text['report_id'].str.extract('_([amr])_')
report_text

## ATSB

In [ ]:
importlib.reload(ReportExtracting)
atsb_reports = report_text[report_text['report_id'].str.contains('ATSB')]
atsb_reports['content_section'] = atsb_reports.apply(
    lambda x: ReportExtracting.ReportExtractor(x['text'], x['report_id']).extract_contents_section(), axis=1
)
atsb_reports

In [ ]:
(atsb_reports['headers'].isnull() & atsb_reports['content_section'].isnull()).value_counts()

In [ ]:
print((atsb_reports['headers'].notna() | atsb_reports['content_section'].notna()).value_counts())

failed = atsb_reports[atsb_reports['content_section'].isnull() & atsb_reports['headers'].isnull()]

(
    ggplot(atsb_reports.assign(
        has_content_section=atsb_reports['content_section'].notna(),
    ), aes(x='year'))
    + geom_histogram()
    + facet_wrap(['mode', 'has_content_section',], ncol=2)
)

ATSB has two sorts of reports. Full investigation and short ones. For the short ones I will not extract the content section. The content section is only used to find the important text to instead I will just provide the wohle text if it is not too long. Most f the short rpeorts are only about 2-5k token which is really not much.

For ATSB full we have 1000 that have been extracted alright.

In [ ]:
atsb_reports['content_section_len'] = atsb_reports['content_section'].map(lambda x: len(x) if x is not None else 0)
atsb_reports.sort_values('content_section_len', ascending=False, inplace=True)

atsb_reports

After looking at this is seems that the content section extraction is accurate. I will add some tests in. Furthermore I will look at what is missed out and see if it is reasonable.

It seems to me that it is reasonable to ignore the failed reports. Most of them are small so the whole report can just be used. For the 50 or so large failed rpeort it can be left for now.

## TSB

TSB is currently running into the issue that it doesn't have content sections for mosts of its reports. I will do ATSB first and will hopefully have a more robust content section extractor. Otherwise I will look into having itgenerate a content sections based on regex searches of the report.

In [ ]:
tsb_reports = report_text[report_text['report_id'].str.startswith('TSB')]

importlib.reload(ReportExtracting)
tsb_reports['content_section'] = tsb_reports.apply(
    lambda x: ReportExtracting.ReportExtractor(x['text'], x['report_id']).extract_contents_section(), axis=1
)
tsb_reports

In [ ]:
failures = tsb_reports['content_section'].isnull() & tsb_reports['headers'].isnull()
print(failures.value_counts())
# Make histogram of years with two colours for content section exist and not
failed = tsb_reports[failures]
(
    ggplot(tsb_reports.assign(has_content_section=tsb_reports['content_section'].notna()), aes(x='year'))
    + geom_histogram()
    + facet_wrap('has_content_section', ncol=1)
)


After finishing with ATSb and we have 1077 False and only 251 true. I will work on this a bit to hopefully get the low hanging fruit.

In [ ]:
tsb_reports['content_section_len'] = tsb_reports['content_section'].map(lambda x: len(x) if x is not None else 0)

tsb_reports.sort_values('content_section_len', ascending=False, inplace=True)

tsb_reports

## TAIC

Even though it is already supported I should know the full information

In [ ]:
taic_reports = report_text[report_text['report_id'].str.startswith('TAIC')]

importlib.reload(ReportExtracting)
taic_reports['content_section'] = taic_reports.apply(
    lambda x: ReportExtracting.ReportExtractor(x['text'], x['report_id']).extract_contents_section(), axis=1
)
taic_reports

In [ ]:
print(taic_reports['content_section'].notna().value_counts())

failed = taic_reports[taic_reports['content_section'].isnull()]

(
    ggplot(taic_reports.assign(has_content_section=taic_reports['content_section'].notna()), aes(x='year'))
    + geom_histogram()
    + facet_wrap('has_content_section', ncol=1)
)

There are 87 failed and 432 true before i start making changing

## Checking the output

At first test there is about 10% failed for TAIC, 50% for ATSB and 16% for TSB.
That is just using the regular expression as it currently is.

We can see that TAIC is only failing once it gets back before 2005 so that can be ignored for now. However both TSB and ATSB have a lot of missing content sections.

In [ ]:
report_id = "ATSB_a_2019_030"
pdf_link = os.path.join(output_folder, 'report_pdfs', f"{report_id}.pdf")
shutil.copy(pdf_link, f"{report_id}.pdf")
text = report_text.query(f'report_id == "{report_id}"').text.values[0]
with open("individual.txt", "w") as f:
    f.write(text)
importlib.reload(ReportExtracting)
parsed_text = ReportExtracting.ReportExtractor(text, report_id).extract_contents_section()

with open("individual-content-section.txt", 'w') as f:
    f.write(parsed_text if parsed_text is not None else '')


for pdf in [pdf for pdf in os.listdir() if pdf.endswith('.pdf')]:
    os.remove(pdf)


display(parsed_text[:15])
display(parsed_text[-15:])
print(len(parsed_text))

In [ ]:
# Save to text files parsed files to let me inspect them
shutil.rmtree('parsed_reports/', ignore_errors=True)

os.makedirs('parsed_reports/', exist_ok=True)
for index, text, content_section in failed[['report_id', 'text', 'content_section']].sample(10, random_state=42).to_records(index=False):
    with open(os.path.join('parsed_reports', f'{index}.txt'), 'w') as f:
        f.write(text)
    with open(os.path.join('parsed_reports', f'{index}_content_section.txt'), 'w') as f:
        f.write(content_section if content_section is not None else '')
    shutil.copy(os.path.join(output_folder, 'report_pdfs', f"{index}.pdf"), os.path.join('parsed_reports', f"{index}.pdf"))

In [ ]:
failed['text_len'] = failed['text'].map(len)
failed.sort_values('text_len', ascending=False, inplace=True)
print(failed['text_len'].describe())
failed

In [ ]:
pd.read_pickle("../../output/important_text.pkl")['important_text'].map(len).describe()

# Looking at PDF outlines

As we can see about 50% of the reports dont have a dedicated content section.

However we can look at more of the pdf details and the outline THis oultline coulh\d help us aas about 2/3 of the rpeorts have an outline.

This has been good and has made ATSB pretty much solved. For TSB we will just go with the default of reading the whole report.

In [ ]:
def process_outline(outline, reader, level =1):
    headers = []
    if not outline:
        return headers
    for item in outline:
        if isinstance(item, list):
            headers.extend(process_outline(item, reader, level + 1))
        else:
            if isinstance(item['/Page'], dict) and '/StructParents' in item['/Page']:
                headers.append({"Title": item["/Title"], "Level": level, "Page": reader.get_destination_page_number(item)})
    
    return headers

In [ ]:
from pypdf import PdfReader
documents = []
for pdf in [pdf for pdf in os.listdir('../../output/report_pdfs') if pdf.endswith('.pdf')]:
    try:
        reader = PdfReader(os.path.join('../../output/report_pdfs', pdf))
    except:
        continue
    documents.append(
        {"report_id": pdf[:-4], "outline": pd.DataFrame(process_outline(reader.outline, reader))}
    )

outline_df = pd.DataFrame(documents)

outline_df.set_index('report_id', inplace=True)

outline_df

In [ ]:
outline_df['agency'] = outline_df.index.str.split('_').map(lambda x: x[0])

outline_df['has_outline'] = outline_df['outline'].map(len) > 0

(
    ggplot(outline_df, aes(x='agency', fill='has_outline'))
    + geom_bar()
)

In [ ]:
headers = outline_df.loc["ATSB_a_2019_030"]['outline']
headers

In [ ]:
test = [
    (0, ''),
    (1, ''),
    (2, ''),
    (3, '1'),
    (4, '2'),
    (5, '3'),
    (6, '4'),
    (7, '5'),
    (8, '6'),
]


test = pd.DataFrame(test).set_index(0)

headers['Page'] = headers['Page'].map(lambda x: test.loc[x][1])
headers

In [ ]:
# Testing out full pdf parsing

import engine.gather.PDFParser as PDFParser
importlib.reload(PDFParser)

PDFParser.convertPDFToText('../../output/report_pdfs', 'converted_with_headers.pkl', False)

In [ ]:
def inspect_headers(row):
    shutil.copy(os.path.join('../../output/report_pdfs', f"{row['report_id']}.pdf"), os.path.join('headers', f"{row['report_id']}.pdf"))
    print(row['report_id'])
    display(row['headers'])

In [ ]:
converted_text = pd.read_pickle('../../output/parsed_reports.pkl')
display(converted_text)
converted_text[converted_text['headers'].notnull()].sample(10, random_state=42).apply(inspect_headers, axis=1)

In [ ]:
importlib.reload(ReportExtracting)
converted_text['content_section'] = [
    ReportExtracting.ReportExtractor(x[1]['text'], x[1]['report_id'], x[1]['headers']).extract_contents_section()
    for x
    in converted_text.iterrows()
    ] 

In [ ]:
converted_text[converted_text['content_section'].map(lambda x: isinstance(x, pd.DataFrame))]

Currently the content section is only used to extract the important text. This important text is then used to extract the safety issues.
Moving forward we can just use the entire report as most of them are not that long. Given that TAIC average important length is 22222 characters we can just do the same for the failed ones as they are on average only 24000 characters.